In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#load traning data
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [3]:
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points
REMOVE_ONLY_2_Outliers = True
USE_DUMMY_CAT_FEATURES = True

In [4]:
df_train = pd.read_csv('./data/train.csv')
df_test =  pd.read_csv('./data/test.csv')
display(df_train.head(5))
display(df_test.head(5))

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.000,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.000,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.000,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.000,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.000,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.000,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.000,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.000,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.000,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.000,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [5]:
#Remove Id field as it is not useful in regression 
df_test_id = df_test['Id']
#print(df_test_id)

df_train.drop(['Id'],axis=1,inplace=True)
df_test.drop(['Id'],axis=1,inplace=True)

#df_train.drop(['Id','PoolQC','MiscFeature','Alley'],axis=1,inplace=True)
#df_test.drop(['Id','PoolQC','MiscFeature','Alley'],axis=1,inplace=True)

print(df_train.shape)
print(df_test.shape)

(1460, 80)
(1459, 79)


In [6]:
# Remove outliers  Reference - https://www.kaggle.com/zoupet/neural-network-model-for-house-prices-tensorflow
from sklearn.ensemble import IsolationForest

def RemoveOutliers(df_train):
    if(REMOVE_ONLY_2_Outliers):
        #Deleting outliers
        df_train = df_train.drop(df_train[(df_train['GrLivArea']>4000) & (df_train['SalePrice']<300000)].index)
    else:
        clf = IsolationForest(max_samples = 100, random_state = 42)
        clf.fit(df_train)
        y_noano = clf.predict(df_train)
        y_noano = pd.DataFrame(y_noano, columns = ['Top'])
        y_noano[y_noano['Top'] == 1].index.values

        df_train = df_train.iloc[y_noano[y_noano['Top'] == 1].index.values]
        df_train.reset_index(drop = True, inplace = True)
        print("Number of Outliers:", y_noano[y_noano['Top'] == -1].shape[0])
        print("Number of rows without outliers:", df_train.shape[0])
        
RemoveOutliers(df_train)

In [7]:
#Log-transformation of the target variable
#Handle skewness(Reference - https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard)
#We use the numpy fuction log1p which  applies log(1+x) to all elements of the column
df_train["SalePrice"] = np.log1p(df_train["SalePrice"])
df_train["SalePrice"].head()

0   12.248
1   12.109
2   12.317
3   11.849
4   12.429
Name: SalePrice, dtype: float64

In [8]:
ntrain = df_train.shape[0]
ntest = df_test.shape[0]
y_train = df_train.SalePrice.values
all_data = pd.concat((df_train, df_test)).reset_index(drop=True)
all_data.drop(['SalePrice'], axis=1, inplace=True)
print("all_data size is : {}".format(all_data.shape))

all_data size is : (2919, 79)


In [9]:
total = all_data.isnull().sum().sort_values(ascending=False)
display(total.head(10))

PoolQC          2909
MiscFeature     2814
Alley           2721
Fence           2348
FireplaceQu     1420
LotFrontage      486
GarageFinish     159
GarageQual       159
GarageYrBlt      159
GarageCond       159
dtype: int64

In [10]:
def datapreprocessing(all_data):
    all_data["PoolQC"] = all_data["PoolQC"].fillna("None")
    all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")
    all_data["Alley"] = all_data["Alley"].fillna("None")
    all_data["Fence"] = all_data["Fence"].fillna("None")
    all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")
    
    #Group by neighborhood and fill in missing value by the median LotFrontage of all the neighborhood
    all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))
    for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
        all_data[col] = all_data[col].fillna('None')
        
    #Replacing missing data with 0 (Since No garage = no cars in such garage.)
    for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
        all_data[col] = all_data[col].fillna(0)
        
    # missing values are likely zero for having no basement
    for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
        all_data[col] = all_data[col].fillna(0)
        
    # For all these categorical basement-related features, NaN means that there is no basement.
    for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
        all_data[col] = all_data[col].fillna('None')
        
    #NA most likely means no masonry veneer for these houses. We can fill 0 for the area and None for the type.
    all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
    all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)
    
    # 'RL' is by far the most common value. So we can fill in missing values with 'RL'
    all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
    
    #For this categorical feature all records are "AllPub", except for one "NoSeWa" and 2 NA . 
    #Since the house with 'NoSewa' is in the training set, this feature won't help in predictive modelling. 
    #We can then safely remove it.
    all_data = all_data.drop(['Utilities'], axis=1)
    
    #data description says NA means typical
    all_data["Functional"] = all_data["Functional"].fillna("Typ")
    
    #It has one NA value. Since this feature has mostly 'SBrkr', we can set that for the missing value.
    all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
    
    #Only one NA value, and same as Electrical, we set 'TA' (which is the most frequent) for the missing value in KitchenQual.
    all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])
    
    #Exterior 1 & 2 have only one missing value. We will just substitute in the most common string
    all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
    all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
    
    all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])
    all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")
    
    #Transforming some numerical variables that are really categorical

    #MSSubClass=The building class
    all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)


    #Changing OverallCond into a categorical variable
    all_data['OverallCond'] = all_data['OverallCond'].astype(str)


    #Year and month sold are transformed into categorical features.
    all_data['YrSold'] = all_data['YrSold'].astype(str)
    all_data['MoSold'] = all_data['MoSold'].astype(str)
    
    
#datapreprocessing(df_train)
#datapreprocessing(df_test)
datapreprocessing(all_data)

all_data.dtypes

1stFlrSF           int64
2ndFlrSF           int64
3SsnPorch          int64
Alley             object
BedroomAbvGr       int64
BldgType          object
BsmtCond          object
BsmtExposure      object
BsmtFinSF1       float64
BsmtFinSF2       float64
BsmtFinType1      object
BsmtFinType2      object
BsmtFullBath     float64
BsmtHalfBath     float64
BsmtQual          object
BsmtUnfSF        float64
CentralAir        object
Condition1        object
Condition2        object
Electrical        object
EnclosedPorch      int64
ExterCond         object
ExterQual         object
Exterior1st       object
Exterior2nd       object
Fence             object
FireplaceQu       object
Fireplaces         int64
Foundation        object
FullBath           int64
                  ...   
LotFrontage      float64
LotShape          object
LowQualFinSF       int64
MSSubClass         int64
MSZoning          object
MasVnrArea       float64
MasVnrType        object
MiscFeature       object
MiscVal            int64


In [11]:
from sklearn.preprocessing import LabelEncoder
def encodeCategories(all_data):
    cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
            'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
            'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
            'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 'YrSold', 'MoSold')
    # process columns, apply LabelEncoder to categorical features
    for c in cols:
        lbl = LabelEncoder() 
        lbl.fit(list(all_data[c].values)) 
        all_data[c] = lbl.transform(list(all_data[c].values))

    # shape        
    print('Shape all_data: {}'.format(all_data.shape))
    #return 
    
#encodeCategories(df_train)
#encodeCategories(df_test)
encodeCategories(all_data)

Shape all_data: (2919, 79)


In [12]:
# Adding total sqfootage feature 
#df_train['TotalSF'] = df_train['TotalBsmtSF'] + df_train['1stFlrSF'] + df_train['2ndFlrSF']
#df_test['TotalSF'] = df_test['TotalBsmtSF'] + df_test['1stFlrSF'] + df_test['2ndFlrSF']
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']

In [13]:
display(all_data.head(5))

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,...,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,TotalSF
0,856,854,0,1,3,1Fam,4,3,706.000,0.000,...,0,1,8,856.000,AllPub,0,2003,2003,2,2566.000
1,1262,0,0,1,3,1Fam,4,1,978.000,0.000,...,0,1,6,1262.000,AllPub,298,1976,1976,1,2524.000
2,920,866,0,1,3,1Fam,4,2,486.000,0.000,...,0,1,6,920.000,AllPub,0,2001,2002,2,2706.000
3,961,756,0,1,3,1Fam,1,3,216.000,0.000,...,0,1,7,756.000,AllPub,0,1915,1970,0,2473.000
4,1145,1053,0,1,4,1Fam,4,0,655.000,0.000,...,0,1,9,1145.000,AllPub,192,2000,2000,2,3343.000


In [14]:
from scipy import stats
from scipy.stats import norm, skew #for some statistics

def skewnesshandling(all_data):
    numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

    # Check the skew of all numerical features
    skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
    print("\nSkew in numerical features: \n")
    skewness = pd.DataFrame({'Skew' :skewed_feats})
    print(skewness.head(10))

    skewness = skewness[abs(skewness) > 0.75]
    print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))
    
    from scipy.special import boxcox1p
    skewed_features = skewness.index
    lam = 0.15
    for feat in skewed_features:
        #all_data[feat] += 1
        all_data[feat] = boxcox1p(all_data[feat], lam)
    
    #all_data[skewed_features] = np.log1p(all_data[skewed_features])
#skewnesshandling(df_train)
#skewnesshandling(df_test)
skewnesshandling(all_data)


Skew in numerical features: 

                Skew
MiscVal       21.947
PoolArea      16.898
LotArea       12.822
LowQualFinSF  12.089
3SsnPorch     11.376
LandSlope      4.975
KitchenAbvGr   4.302
BsmtFinSF2     4.146
EnclosedPorch  4.004
ScreenPorch    3.947
There are 59 skewed numerical features to Box Cox transform


In [15]:
# Separate numerical and categorical columns
def getNumericaldata(df,excludefield):
    quantitative = [f for f in df.columns if df.dtypes[f] != 'object']
    quantitative.remove(excludefield)
    return quantitative

def getCategoricaldata(df):
    qualitative = [f for f in df.columns if df.dtypes[f] == 'object']
    return qualitative

numcols = getNumericaldata(df_train,'SalePrice')
catcols = getCategoricaldata(df_train)

print(numcols)
print(catcols)

['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCon

In [16]:
def convertasCategory(df,cols):
    for col in cols:
        df[col] = df[col].astype('category')
        df[col+'_code'] = df[col].cat.codes
        df[col] = df[col +'_code']
        df.drop(labels=col+'_code', axis="columns", inplace=True)
    #return df

if(USE_DUMMY_CAT_FEATURES):
    #df_train = pd.get_dummies(df_train)
    #print(df_train.shape)
    #df_test = pd.get_dummies(df_test)
    #print(df_test.shape)
    all_data = pd.get_dummies(all_data)
    print(all_data.shape)
else:
    convertasCategory(df_train,catcols)
    convertasCategory(df_test,catcols)

(2919, 223)


In [17]:
   

#print(df_train.head())
#print(df_train.dtypes)
# Handle remaining missing values for numerical features by using median as replacement
#print("NAs for numerical features in train : " + str(df_train.isnull().values.sum()))
######df_train = df_train.fillna(df_train.median())
######df_test = df_test.fillna(df_train.median())
#print("Remaining NAs for numerical features in train : " + str(df_train.isnull().values.sum()))
#print("Remaining NAs for numerical features in test : " + str(df_test.isnull().values.sum()))
print("Remaining NAs for numerical features in all_data : " + str(all_data.isnull().values.sum()))

Remaining NAs for numerical features in all_data : 0


In [18]:
df_train = all_data[:ntrain]
df_test = all_data[ntrain:]
print(df_train.shape)
print(df_test.shape)

(1460, 223)
(1459, 223)


In [19]:
from sklearn.model_selection import train_test_split


X = df_train.iloc[:, 1:-2].values  
#print(X[1]) 
print(X.shape)
y = y_train ##df_train.iloc[:, -1].values
#print(y)

X_test = df_test.iloc[:, 1:-2].values  
#print(X_test[1]) 
print(X_test.shape)
y_test = df_test.iloc[:, -1].values
print(y_test)


X_train, X_train_test, y_train, y_train_test = train_test_split(X, y, test_size=0.1, random_state=101)


(1460, 220)
(1459, 220)
[0 0 0 ... 0 0 0]


In [20]:
# we are going to scale to data

y_train= y_train.reshape(-1,1)
y_train_test= y_train_test.reshape(-1,1)
y_test= y_train_test.reshape(-1,1)

from sklearn.preprocessing import StandardScaler
sc_X_train = StandardScaler()
sc_y_train = StandardScaler()

sc_X_train_test = StandardScaler()
sc_y_train_test = StandardScaler()

sc_X_test = StandardScaler()
sc_y_test = StandardScaler()


X_train = sc_X_train.fit_transform(X_train)
y_train = sc_y_train.fit_transform(y_train)

X_train_test = sc_X_train_test.fit_transform(X_train_test)
y_train_test = sc_y_train_test.fit_transform(y_train_test)

X_test = sc_X_test.fit_transform(X_test)
y_test = sc_y_test.fit_transform(y_test)

In [21]:
import xgboost as xgb
from xgboost import XGBRegressor
#xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
#                max_depth = 5, alpha = 10, n_estimators = 1000)
eval_set = [(X_train, y_train), (X_train_test, y_train_test)]
eval_metric = ["rmse","error"]
xg_reg = xgb.XGBRegressor(colsample_bytree=0.4, gamma=0,learning_rate=0.07, max_depth=3, min_child_weight=1.5,
                 n_estimators=10000, reg_alpha=0.75, reg_lambda=0.45,subsample=0.6, seed=42) 
#xg_reg = xgb.XGBRegressor(gamma=1, colsample_bytree = .5, learning_rate = 0.1,
#                max_depth = 3, alpha = 10, n_estimators = 1000)
#xgbmodel = xg_reg.fit(X,y,verbose=True)
xgbmodel = xg_reg.fit(X_train, y_train, eval_metric=eval_metric, eval_set=eval_set, verbose=True)
print(xgbmodel.feature_importances_)
print("?????????????????????????????")
#X, y = make_regression(n_features=4, n_informative=2,random_state=0, shuffle=False)
#regr = RandomForestRegressor(max_depth=2, random_state=0)
#regr.fit(X, y)
#RandomForestRegressor(max_depth=2, random_state=0)
#print(regr.feature_importances_)
#print(regr.feature_importances_.shape)
#print(X.shape)
Xcols=df_train.columns
result = zip(Xcols,xgbmodel.feature_importances_.tolist())
df_impf = pd.DataFrame(set(result))
df_impf.columns = ['field','score']
print(df_impf.sort_values('score',ascending=False))


[17:17:50] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:1.05737	validation_0-error:-0.322611	validation_1-rmse:1.05978	validation_1-error:-0.335145
[1]	validation_0-rmse:1.00123	validation_0-error:-0.357295	validation_1-rmse:1.00798	validation_1-error:-0.338842
[2]	validation_0-rmse:0.952724	validation_0-error:-0.364429	validation_1-rmse:0.960807	validation_1-error:-0.34927
[3]	validation_0-rmse:0.906736	validation_0-error:-0.363454	validation_1-rmse:0.916401	validation_1-error:-0.340422
[4]	validation_0-rmse:0.863574	validation_0-error:-0.364421	validation_1-rmse:0.884095	validation_1-error:-0.341386
[5]	validation_0-rmse:0.822351	validation_0-error:-0.362807	validation_1-rmse:0.847098	validation_1-error:-0.339111
[6]	validation_0-rmse:0.784239	validation_0-error:-0.362919	validation_1-rmse:0.812306	validation_1-error:-0.339111
[7]	validation_0-rmse:0.753446	validation_0-error:-0.364137	validation_1-rmse

[70]	validation_0-rmse:0.271107	validation_0-error:-0.385798	validation_1-rmse:0.357627	validation_1-error:-0.356441
[71]	validation_0-rmse:0.270139	validation_0-error:-0.385798	validation_1-rmse:0.35676	validation_1-error:-0.356441
[72]	validation_0-rmse:0.268716	validation_0-error:-0.38558	validation_1-rmse:0.35537	validation_1-error:-0.356441
[73]	validation_0-rmse:0.267755	validation_0-error:-0.386184	validation_1-rmse:0.35458	validation_1-error:-0.356441
[74]	validation_0-rmse:0.266705	validation_0-error:-0.386966	validation_1-rmse:0.35313	validation_1-error:-0.356441
[75]	validation_0-rmse:0.265487	validation_0-error:-0.387439	validation_1-rmse:0.352508	validation_1-error:-0.356441
[76]	validation_0-rmse:0.264938	validation_0-error:-0.3874	validation_1-rmse:0.351681	validation_1-error:-0.356441
[77]	validation_0-rmse:0.264164	validation_0-error:-0.3874	validation_1-rmse:0.35065	validation_1-error:-0.356441
[78]	validation_0-rmse:0.262978	validation_0-error:-0.3874	validation_1-rm

[140]	validation_0-rmse:0.226127	validation_0-error:-0.393524	validation_1-rmse:0.324534	validation_1-error:-0.356441
[141]	validation_0-rmse:0.225768	validation_0-error:-0.393699	validation_1-rmse:0.324609	validation_1-error:-0.356441
[142]	validation_0-rmse:0.225471	validation_0-error:-0.393699	validation_1-rmse:0.324334	validation_1-error:-0.356441
[143]	validation_0-rmse:0.225176	validation_0-error:-0.393699	validation_1-rmse:0.32425	validation_1-error:-0.356441
[144]	validation_0-rmse:0.224753	validation_0-error:-0.393699	validation_1-rmse:0.323116	validation_1-error:-0.356441
[145]	validation_0-rmse:0.224233	validation_0-error:-0.393699	validation_1-rmse:0.32289	validation_1-error:-0.356441
[146]	validation_0-rmse:0.22392	validation_0-error:-0.393699	validation_1-rmse:0.322789	validation_1-error:-0.356441
[147]	validation_0-rmse:0.223629	validation_0-error:-0.393699	validation_1-rmse:0.322874	validation_1-error:-0.356441
[148]	validation_0-rmse:0.223334	validation_0-error:-0.3936

[210]	validation_0-rmse:0.203681	validation_0-error:-0.394882	validation_1-rmse:0.310207	validation_1-error:-0.356573
[211]	validation_0-rmse:0.203501	validation_0-error:-0.394861	validation_1-rmse:0.310067	validation_1-error:-0.356573
[212]	validation_0-rmse:0.203136	validation_0-error:-0.395048	validation_1-rmse:0.309795	validation_1-error:-0.356573
[213]	validation_0-rmse:0.202682	validation_0-error:-0.395153	validation_1-rmse:0.309229	validation_1-error:-0.356573
[214]	validation_0-rmse:0.202417	validation_0-error:-0.394835	validation_1-rmse:0.308944	validation_1-error:-0.356573
[215]	validation_0-rmse:0.202227	validation_0-error:-0.395153	validation_1-rmse:0.308966	validation_1-error:-0.356573
[216]	validation_0-rmse:0.201931	validation_0-error:-0.395562	validation_1-rmse:0.308417	validation_1-error:-0.356573
[217]	validation_0-rmse:0.201471	validation_0-error:-0.395545	validation_1-rmse:0.308454	validation_1-error:-0.356573
[218]	validation_0-rmse:0.201161	validation_0-error:-0.3

[280]	validation_0-rmse:0.186977	validation_0-error:-0.3959	validation_1-rmse:0.303541	validation_1-error:-0.356573
[281]	validation_0-rmse:0.186653	validation_0-error:-0.3959	validation_1-rmse:0.303186	validation_1-error:-0.356573
[282]	validation_0-rmse:0.18634	validation_0-error:-0.395752	validation_1-rmse:0.303089	validation_1-error:-0.356573
[283]	validation_0-rmse:0.186133	validation_0-error:-0.396033	validation_1-rmse:0.3031	validation_1-error:-0.356573
[284]	validation_0-rmse:0.185961	validation_0-error:-0.396033	validation_1-rmse:0.303098	validation_1-error:-0.356573
[285]	validation_0-rmse:0.185576	validation_0-error:-0.396033	validation_1-rmse:0.303502	validation_1-error:-0.356573
[286]	validation_0-rmse:0.185376	validation_0-error:-0.395752	validation_1-rmse:0.303332	validation_1-error:-0.356573
[287]	validation_0-rmse:0.18532	validation_0-error:-0.3959	validation_1-rmse:0.303332	validation_1-error:-0.356573
[288]	validation_0-rmse:0.185107	validation_0-error:-0.396181	vali

...

[9723]	validation_0-rmse:0.032977	validation_0-error:-0.404989	validation_1-rmse:0.298603	validation_1-error:-0.356573
[9724]	validation_0-rmse:0.032977	validation_0-error:-0.404989	validation_1-rmse:0.298603	validation_1-error:-0.356573
[9725]	validation_0-rmse:0.032974	validation_0-error:-0.404989	validation_1-rmse:0.2986	validation_1-error:-0.356573
[9726]	validation_0-rmse:0.032971	validation_0-error:-0.404989	validation_1-rmse:0.298606	validation_1-error:-0.356573
[9727]	validation_0-rmse:0.03297	validation_0-error:-0.404989	validation_1-rmse:0.298605	validation_1-error:-0.356573
[9728]	validation_0-rmse:0.032968	validation_0-error:-0.404989	validation_1-rmse:0.298608	validation_1-error:-0.356573
[9729]	validation_0-rmse:0.032965	validation_0-error:-0.404989	validation_1-rmse:0.298596	validation_1-error:-0.356573
[9730]	validation_0-rmse:0.032964	validation_0-error:-0.404989	validation_1-rmse:0.298591	validation_1-error:-0.356573
[9731]	validation_0-rmse:0.032963	validation_0-erro

[9792]	validation_0-rmse:0.032898	validation_0-error:-0.404989	validation_1-rmse:0.298602	validation_1-error:-0.356573
[9793]	validation_0-rmse:0.032898	validation_0-error:-0.404989	validation_1-rmse:0.298602	validation_1-error:-0.356573
[9794]	validation_0-rmse:0.032895	validation_0-error:-0.404989	validation_1-rmse:0.29861	validation_1-error:-0.356573
[9795]	validation_0-rmse:0.03289	validation_0-error:-0.404989	validation_1-rmse:0.298631	validation_1-error:-0.356573
[9796]	validation_0-rmse:0.032887	validation_0-error:-0.404989	validation_1-rmse:0.298634	validation_1-error:-0.356573
[9797]	validation_0-rmse:0.032886	validation_0-error:-0.404989	validation_1-rmse:0.298636	validation_1-error:-0.356573
[9798]	validation_0-rmse:0.032883	validation_0-error:-0.404989	validation_1-rmse:0.298643	validation_1-error:-0.356573
[9799]	validation_0-rmse:0.032881	validation_0-error:-0.404989	validation_1-rmse:0.298639	validation_1-error:-0.356573
[9800]	validation_0-rmse:0.032878	validation_0-err

[9861]	validation_0-rmse:0.032804	validation_0-error:-0.404989	validation_1-rmse:0.298584	validation_1-error:-0.356573
[9862]	validation_0-rmse:0.032803	validation_0-error:-0.404989	validation_1-rmse:0.298594	validation_1-error:-0.356573
[9863]	validation_0-rmse:0.0328	validation_0-error:-0.404989	validation_1-rmse:0.298587	validation_1-error:-0.356573
[9864]	validation_0-rmse:0.0328	validation_0-error:-0.404989	validation_1-rmse:0.298587	validation_1-error:-0.356573
[9865]	validation_0-rmse:0.032799	validation_0-error:-0.404989	validation_1-rmse:0.29858	validation_1-error:-0.356573
[9866]	validation_0-rmse:0.032798	validation_0-error:-0.404989	validation_1-rmse:0.298577	validation_1-error:-0.356573
[9867]	validation_0-rmse:0.032797	validation_0-error:-0.404989	validation_1-rmse:0.298583	validation_1-error:-0.356573
[9868]	validation_0-rmse:0.032795	validation_0-error:-0.404989	validation_1-rmse:0.298595	validation_1-error:-0.356573
[9869]	validation_0-rmse:0.032793	validation_0-error:

[9930]	validation_0-rmse:0.032715	validation_0-error:-0.404989	validation_1-rmse:0.298579	validation_1-error:-0.356573
[9931]	validation_0-rmse:0.032713	validation_0-error:-0.404989	validation_1-rmse:0.298585	validation_1-error:-0.356573
[9932]	validation_0-rmse:0.03271	validation_0-error:-0.404989	validation_1-rmse:0.298585	validation_1-error:-0.356573
[9933]	validation_0-rmse:0.032708	validation_0-error:-0.404989	validation_1-rmse:0.298576	validation_1-error:-0.356573
[9934]	validation_0-rmse:0.032708	validation_0-error:-0.404989	validation_1-rmse:0.298582	validation_1-error:-0.356573
[9935]	validation_0-rmse:0.032708	validation_0-error:-0.404989	validation_1-rmse:0.298582	validation_1-error:-0.356573
[9936]	validation_0-rmse:0.032707	validation_0-error:-0.404989	validation_1-rmse:0.298598	validation_1-error:-0.356573
[9937]	validation_0-rmse:0.032705	validation_0-error:-0.404989	validation_1-rmse:0.298571	validation_1-error:-0.356573
[9938]	validation_0-rmse:0.032703	validation_0-er

[9999]	validation_0-rmse:0.032627	validation_0-error:-0.404989	validation_1-rmse:0.298596	validation_1-error:-0.356573
[1.65783614e-03 5.44641691e-04 8.12673999e-04 9.89539782e-04
 1.51389255e-03 2.72527221e-03 5.71163185e-03 7.06329476e-04
 2.47430778e-03 9.34626441e-04 3.76595720e-03 8.84824898e-04
 6.09195558e-03 1.20988139e-03 4.69534248e-02 1.35773455e-03
 1.69284514e-03 6.62821680e-02 8.67915747e-04 7.03183643e-04
 2.18490176e-02 5.62236644e-03 8.97176471e-03 6.02571946e-03
 6.46353811e-02 3.21413530e-03 9.26035049e-04 8.59837700e-03
 3.33600072e-03 1.70796514e-02 3.05626099e-03 3.15891579e-03
 2.22861930e-03 3.34141999e-02 9.30104696e-04 3.52365617e-03
 1.67137478e-03 2.24097469e-03 1.47333587e-04 1.02199242e-03
 7.79670605e-04 2.74159451e-04 9.16033227e-04 2.18143617e-03
 1.14512704e-02 2.18911067e-01 4.12287284e-03 1.31739406e-02
 0.00000000e+00 2.02619983e-03 0.00000000e+00 1.04260724e-03
 3.24793509e-03 1.30527851e-03 1.19362045e-02 6.14396948e-03
 9.03305830e-04 6.12706020e

In [22]:
predictions = xgbmodel.predict(X_test)
print(predictions)
print(predictions.shape)
predictions = np.expm1(sc_y_test.inverse_transform(predictions))
#predictions = sc_y_test.inverse_transform(predictions)
df_predictions = pd.DataFrame(predictions, columns=['SalePrice'])
#df_predictions.head()

#submission data
df_sub1 = pd.concat([df_test_id,df_predictions],axis=1)
print(df_sub1.head())
df_sub1.to_csv('./data/submission1.csv',index=False)

[-0.71328545 -0.11883968  0.26470077 ...  0.0907346  -0.8391063
  0.66839576]
(1459,)
     Id  SalePrice
0  1461 125468.336
1  1462 157959.219
2  1463 183262.094
3  1464 192963.844
4  1465 175292.812


In [23]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics
import numpy as np

svr = SVR(kernel = 'rbf',gamma='auto')
svr.fit(X_train, y_train)

svr_pred = sc_y_test.inverse_transform(svr.predict(X_train_test))
svr_pred= svr_pred.reshape(-1,1)

print('MAE:', metrics.mean_absolute_error(y_train_test, svr_pred))
print('MSE:', metrics.mean_squared_error(y_train_test, svr_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_train_test, svr_pred)))

MAE: 12.028491346069405
MSE: 145.2076493596859
RMSE: 12.050213664482714


In [24]:
from sklearn import ensemble
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, r2_score

params = {'n_estimators': 500, 'max_depth': 4, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}
params1={'n_estimators':3000, 'learning_rate':0.05, 'max_depth':4, 'max_features':'sqrt',
                                   'min_samples_leaf':15, 'min_samples_split':2, 
                                   'loss':'huber', 'random_state':5}
clf = ensemble.GradientBoostingRegressor(**params1)

clf.fit(X_train, y_train)

clf_pred = sc_y_train_test.inverse_transform(clf.predict(X_train_test))
clf_pred = clf_pred.reshape(-1,1)

print('MAE:', metrics.mean_absolute_error(y_train_test, clf_pred))
print('MSE:', metrics.mean_squared_error(y_train_test, clf_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_train_test, clf_pred)))



MAE: 12.010447046904
MSE: 144.7211839671658
RMSE: 12.030011802453306


In [25]:
clf_pred = np.expm1(clf_pred)
#print(clf_pred)
df_clf_pred = pd.DataFrame(clf_pred, columns=['SalePrice'])
df_clf_pred.head()

,SalePrice
0,251680.284
1,146841.577
2,145566.763
3,385027.714
4,386855.861


In [26]:
#try gbr on test data
clf_pred=clf.predict(X_test)
clf_pred= clf_pred.reshape(-1,1)
clf_pred = np.expm1(sc_y_test.inverse_transform(clf_pred))
#print(clf_pred)
df_clf_pred = pd.DataFrame(clf_pred, columns=['SalePrice'])
df_clf_pred.head()

,SalePrice
0,124516.012
1,155406.279
2,190034.879
3,196456.921
4,168468.348


In [27]:
#submission data
df_sub = pd.concat([df_test_id,df_clf_pred],axis=1)
print(df_sub.head())

     Id  SalePrice
0  1461 124516.012
1  1462 155406.279
2  1463 190034.879
3  1464 196456.921
4  1465 168468.348


In [28]:
df_sub.to_csv('./data/submission.csv',index=False)

Change Log
++++++++++++++++++++++++++++++++++=

Feartue Engineering
Replace missing values by median

Model
SVR

Score
MAE: 50138.725050989204
    
MSE: 5299712145.626122
    
RMSE: 72799.12187400424

+++++++++++++++++++++++++++++++++
Params1
MAE: 0.21022910696011998
MSE: 0.10793411607512507
RMSE: 0.3285332800115158
    
Params
MAE: 0.21493207760478975
MSE: 0.11624055545701495
RMSE: 0.3409406919935122



In [29]:
#Try out encoding as done here - https://www.kaggle.com/yingbao/feature-engineering-for-house-price-prediction
# Change the categorical data to numerical data. Avoid using get_dummies at this stage, because it will generate too many variables.
# function to add an encoded column for each categorical variable
"""
def encode(frame, feature):
    ordering = pd.DataFrame()
    # add a column of 'val' to dataframe. values in 'val' is the values in a specific categorical column
    ordering['val'] = frame[feature].unique()
    print(ordering['val'])
    # add index
    ordering.index = ordering.val
    # spmean is based on the average SalePrice for each categorical value (group by each categorical value)
    ordering['spmean'] = frame[[feature, 'SalePrice']].groupby(feature).mean()['SalePrice']
    print("*************************************")
    print(ordering['spmean'])
    # sort according to the spmean and rank it
    ordering = ordering.sort_values('spmean')
    ordering['ordering'] = range(1, ordering.shape[0]+1)
    # convert it to a dictionary
    ordering = ordering['ordering'].to_dict()
    print("^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
    print(ordering)
    print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
    # the a new column _E to the orginal df_train. 
    for cat, o in ordering.items():
        print(cat)
        print(o)
        frame.loc[frame[feature]==cat, feature + '_E'] = o
        
# the new columns provide quick influence estimations of categorical variable on SalePrice
# here, we just want to get a rough idea, so consider other variables as fixed
qual_encoded = []
qualitative=catcols
for q in qualitative:
    encode(df_train, q)
    qual_encoded.append(q+'_E')
print(qual_encoded)
"""

'\ndef encode(frame, feature):\n    ordering = pd.DataFrame()\n    # add a column of \'val\' to dataframe. values in \'val\' is the values in a specific categorical column\n    ordering[\'val\'] = frame[feature].unique()\n    print(ordering[\'val\'])\n    # add index\n    ordering.index = ordering.val\n    # spmean is based on the average SalePrice for each categorical value (group by each categorical value)\n    ordering[\'spmean\'] = frame[[feature, \'SalePrice\']].groupby(feature).mean()[\'SalePrice\']\n    print("*************************************")\n    print(ordering[\'spmean\'])\n    # sort according to the spmean and rank it\n    ordering = ordering.sort_values(\'spmean\')\n    ordering[\'ordering\'] = range(1, ordering.shape[0]+1)\n    # convert it to a dictionary\n    ordering = ordering[\'ordering\'].to_dict()\n    print("^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")\n    print(ordering)\n    print("<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")\n    # the a new column _E to the orgina

In [30]:
### References
# Remove Outliers - https://www.kaggle.com/zoupet/neural-network-model-for-house-prices-tensorflow 
# GridSearchCV - https://medium.com/datadriveninvestor/an-introduction-to-grid-search-ff57adcc0998
# Skewness - https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard
#
#